# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [2]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 667.8/667.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 14.6 M

In [3]:
url = "https://storage.googleapis.com/qm2/CASA0025/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://storage.googleapis.com/qm2/CASA0025/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 66.0MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [4]:
con = duckdb.connect("nyc_data.db")

In [5]:
con.install_extension("spatial")
con.load_extension("spatial")

In [6]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

4. **What neighborhood has the highest population density (persons/km2)?**


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [19]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.6/194.6 kB 17.7 MB/s eta 0:00:00


In [20]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [21]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [22]:
%%sql
-- Create a temporary table from the CSV data
CREATE OR REPLACE TEMPORARY TABLE raw_ais_data AS
FROM 'https://storage.googleapis.com/qm2/casa0025_ships.csv';

-- Create the 'ais' table with AIS ping data and a geometry column
CREATE OR REPLACE TABLE ais AS
SELECT
    vesselid,
    vessel_name,
    vsl_descr,
    dwt,
    v_length,
    draught,
    sog,
    date,
    lat,
    lon,
    ST_GeomFromText(CONCAT('POINT (', lon, ' ', lat, ')')) AS geom
FROM raw_ais_data;

-- Create a spatial index on the 'ais' table
CREATE INDEX ais_geom_idx ON ais USING RTREE(geom);

-- Create the 'vinfo' table with unique vessel-level information
CREATE OR REPLACE TABLE vinfo AS
SELECT DISTINCT
    vesselid,
    vessel_name,
    vsl_descr,
    dwt,
    v_length
FROM raw_ais_data;

,Success


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [23]:
%%sql
WITH ClosestPings AS (
    SELECT
        a.vesselid AS vesselid1,
        a.vessel_name AS vessel_name1,
        b.vesselid AS vesselid2,
        b.vessel_name AS vessel_name2,
        LEAST(a.date, b.date) AS event_start_time, -- Use the earlier timestamp as the reference for the event
        a.sog AS sog1,
        b.sog AS sog2
    FROM
        ais AS a
    JOIN
        ais AS b ON a.vesselid < b.vesselid -- Only get unique pairs (vessel1, vessel2) once and avoid self-join
    WHERE
        ST_DWithin(a.geom, b.geom, 500) -- Within 500 meters
        AND a.sog < 1
        AND b.sog < 1
        -- Pings must be very close in time to represent the same "moment" of proximity
        AND ABS(EPOCH(a.date) - EPOCH(b.date)) <= 600 -- within 10 minutes (600 seconds)
),
GroupedEvents AS (
    SELECT
        vesselid1,
        vesselid2,
        vessel_name1,
        vessel_name2,
        event_start_time,
        LAG(event_start_time) OVER (PARTITION BY vesselid1, vesselid2 ORDER BY event_start_time) AS prev_event_start_time
    FROM
        ClosestPings
),
TransferBlocks AS (
    SELECT
        vesselid1,
        vesselid2,
        vessel_name1,
        vessel_name2,
        event_start_time,
        prev_event_start_time,
        -- Assign a new block_id if the gap between consecutive pings is more than 30 minutes (1800 seconds)
        SUM(CASE WHEN EPOCH(event_start_time) - EPOCH(prev_event_start_time) > 1800 OR prev_event_start_time IS NULL THEN 1 ELSE 0 END) OVER (PARTITION BY vesselid1, vesselid2 ORDER BY event_start_time) AS block_id
    FROM
        GroupedEvents
),
BlockDurations AS (
    SELECT
        vesselid1,
        vesselid2,
        vessel_name1,
        vessel_name2,
        MIN(event_start_time) AS block_start,
        MAX(event_start_time) AS block_end,
        (EPOCH(MAX(event_start_time)) - EPOCH(MIN(event_start_time))) AS duration_seconds
    FROM
        TransferBlocks
    GROUP BY
        vesselid1, vesselid2, vessel_name1, vessel_name2, block_id
)
SELECT
    vesselid1,
    vessel_name1,
    vesselid2,
    vessel_name2,
    block_start,
    block_end,
    duration_seconds / 3600.0 AS duration_hours
FROM
    BlockDurations
WHERE
    duration_seconds >= 7200 -- More than 2 hours (7200 seconds)
ORDER BY
    duration_hours DESC, vesselid1, vesselid2, block_start;

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,vesselid1,vessel_name1,vesselid2,vessel_name2,block_start,block_end,duration_hours
